In [1]:
import requests
import json
from bs4 import BeautifulSoup
import config
import datetime
from datetime import timedelta

# Urls con api news y categorías con Beautiful Soup
El resultado es una lista "seleccionadas" con 5 notas por categoría

In [248]:
mydate = datetime.date(2018,9, 21) 
secciones = ['economia', 'deportes-2', 'sociedad', 'politica', 'teleshow/infoshow', 'america/mundo']
#también tiene entretenimiento, cual es la de internacionales, america + algo, mundo??

In [249]:
def obtener_urls(url):
    response = requests.get(url)
    google_soup = BeautifulSoup(response.text, 'html.parser' )
    urls = []
    for h3 in google_soup.find_all('h3'):
        for a in h3.find_all('a'):
            urls.append(a['href'].replace('/url?q=', ''))
    urls = [link for link in urls if seccion + '/' + año + '/' + mes + '/' +  dia + '/' in link]
    return urls

In [250]:
def obtener_nota(url):
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser' )
    titular = html_soup.find('h1').text
    try:
        autor= html_soup.find('a', {'class':'author-name'}).text
    except:
        autor = ""
    comentarios = bool(html_soup.find_all('h2', text = 'Comentarios'))
    cuerpo = ""
    for div in html_soup.find_all('div', {'id': 'article-body'}):
        for p in div.find_all('p'):
            cuerpo += p.text
    import re
    cuerpo = re.sub(r'Seguí leyendo.+', '', cuerpo).strip()
    cuerpo.replace('\xa0', ' ')
    nota = {'titulo': titular,
       'autor':autor,
       'cuerpo':cuerpo,
       'url':url,
       'fecha':fecha,
       'seccion': seccion,
       'abierta_comentarios': comentarios}
    return nota

In [246]:
%%time
notas = []
for dia in range(31):
    fecha = mydate.strftime("%d/%m/%y")
    año = '20' + mydate.strftime("%y")
    mes = mydate.strftime("%m")
    dia = mydate.strftime("%d")
    print('\n\n' + fecha)
    mydate += timedelta(days=1)
    for seccion in secciones:
        url = 'https://www.google.com.ar/search?ei=SlfLW6OEO8XI5gLvl5ngAg&q=*+site%3Ahttps%3A%2F%2Fwww.infobae.com%2F' + seccion + '%2F' + año + '%2F' + mes +'%2F' +dia+ '%2F&oq=*+site%3Ahttps%3A%2F%2Fwww.infobae.com%2F' + categ + '%2F' + año + '%2F' +mes+ '%2F' + dia + '%2F&gs_l=psy-ab.4...21974.22911.0.23438.4.4.0.0.0.0.260.513.2-2.2.0....0...1.1.64.psy-ab..2.0.0....0.xnbtghTKDls'
        urls = obtener_urls(url)
        print('\n' + seccion + ': ' + str(len(urls)))
        indice = 0
        while indice < 5:
            print(indice, end = ' ')
            if indice == len(urls):
                break
            notas.append(obtener_nota(urls[indice]))
            indice +=1


25/09/18

economia: 10
0 1 2 3 4 
deportes-2: 10
0 1 2 3 4 
sociedad: 8
0 1 2 3 4 
politica: 10
0 1 2 3 4 
teleshow/infoshow: 10
0 1 2 3 4 
america/mundo: 10
0 1 2 3 4 
26/09/18

economia: 10
0 1 2 3 4 
deportes-2: 10
0 1 2 3 4 
sociedad: 10
0 1 2 3 4 
politica: 10
0 1 2 3 4 
teleshow/infoshow: 10
0 1 2 3 4 
america/mundo: 10
0 1 2 3 4 Wall time: 1min 28s


In [253]:
pd.DataFrame(notas).to_csv('./data/infobae_notas.csv')

# Selenium para conseguir comments

In [223]:
import pandas as pd
df = pd.read_csv('./data/infobae_notas.csv', index_col = 0)

In [224]:
df.head(5)

,abierta_comentarios,autor,cuerpo,fecha,seccion,titulo,url
0,True,NaN,"Sorpresa: Luis Caputo, presentó este martes al...",25/09/18,economia,Luis Caputo renunció al Banco Central,https://www.infobae.com/economia/2018/09/25/lu...
1,True,NaN,"El actual secretario de Política Económica, Gu...",25/09/18,economia,Guido Sandleris es el reemplazante de Caputo e...,https://www.infobae.com/economia/2018/09/25/gu...
2,True,NaN,El crecimiento más rápido de Uber en todo el m...,25/09/18,economia,Argentina registró el mayor crecimiento de Ube...,https://www.infobae.com/economia/2018/09/25/ar...
3,True,NaN,Con la llegada de Guido Sandleris a la preside...,25/09/18,economia,"Quién es Verónica Rappoport, la economista que...",https://www.infobae.com/economia/2018/09/25/qu...
4,True,Román Lejtman,(Enviado especial a EEUU) El presidente Mauric...,25/09/18,economia,Mauricio Macri se reúne con Christine Lagarde ...,https://www.infobae.com/economia/2018/09/25/ma...


In [147]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

path_to_extension = r'C:\Users\p_bag\OneDrive\Documentos\1.17.0_0'
path_to_extension_linux = r'/home/pablo/Documentos/1.17.0_0'
windows = '/Users/p_bag/Downloads/chromedriver_win32/chromedriver.exe'
linux = '/home/pablo/Descargas/chromedriver'

chrome_options = Options()
chrome_options.add_argument('load-extension=' + path_to_extension)


#driver = webdriver.Chrome(windows, chrome_options=chrome_options)
#driver.create_options()


In [217]:
comentarios = []
def scrapear(urls):
    driver = webdriver.Chrome(windows, chrome_options=chrome_options)
    driver.create_options()
    indice = 0
    #for url in urls:
    for indice, url in zip(urls.index, urls.values):
        print (indice)
        print(url)
        driver.get(url)
        #time.sleep(1)
        #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        iframe = driver.find_element_by_xpath("//iframe[@title='fb:comments Facebook Social Plugin']")
        #iframe = driver.find_element_by_xpath("//*[@id='f0dmkLDVnJzT5r']/div[1]/span/iframe")
        driver.switch_to.frame(iframe)
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'button')))
            print ("Hay boton")
            button = True
        except TimeoutException:
            print ("No hay boton")
            button = False
        #Que pasa si la nota esta cerrada a comentarios
        while button:
            try:
                #print('boton')
                element = driver.find_element_by_tag_name('button')
                element.location_once_scrolled_into_view
                #time.sleep(2)
                driver.find_element_by_xpath("//button").click()
                WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.TAG_NAME, 'button')))

            except: 
                button = False
        driver.switch_to.parent_frame()
        driver.execute_script("window.scrollTo(0, 0)") 
        driver.switch_to.frame(iframe)
        button = True
        while button:
            try:
                element = driver.find_element_by_xpath("//span[contains(text(), 'Mostrar')]")
                element.location_once_scrolled_into_view
                driver.find_element_by_xpath("//span[contains(text(), 'Mostrar')]").click()
                time.sleep(0.5)
                WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Mostrar')]")))

            except:
                button = False
        todo = driver.find_elements_by_class_name('_5nz1')
        for item in todo:
            #es_respuesta = bool(item.find_elements_by_class_name('_3-8y'))
            #print(item.find_element_by_class_name('_5mdd').text)
            print('.', end = '')
            comentarios.append({
                'autor': item.find_element_by_class_name('UFICommentActorName').text,
                'texto': item.find_element_by_class_name('_5mdd').text,
                'fecha': item.find_element_by_class_name('_2vq9').find_elements_by_tag_name('span')[-1].text,
                'likes': item.find_element_by_class_name('_2vq9').find_elements_by_tag_name('span')[-3].text,
                'respuestas' : len(item.find_elements_by_class_name('_3-8y')),
                'es_respuesta': False,
                'nota_id': indice,
                'nota_url': url
                })
            for respuesta in item.find_elements_by_class_name('_3-8y'):
                #print(respuesta.find_element_by_class_name('_5mdd').text)
                print('.', end = '')
                comentarios.append({
                    'autor': respuesta.find_element_by_class_name('UFICommentActorName').text,
                    'texto': respuesta.find_element_by_class_name('_5mdd').text,
                    'fecha': respuesta.find_element_by_class_name('_2vq9').find_elements_by_tag_name('span')[-1].text,
                    'likes': respuesta.find_element_by_class_name('_2vq9').find_elements_by_tag_name('span')[-3].text,
                    'respuestas' : len(respuesta.find_elements_by_class_name('_3-8y')),
                    'es_respuesta': True,
                    'nota_id': indice,
                    'nota_url': url
                     })
    return comentarios

In [225]:
#df2 = df.copy()
#df = df[:10]
#df.shape[0]

In [234]:
corte = df.url.shape[0]//4
lista_urls = []
lista_urls.append(df.url[:corte])
lista_urls.append(df.url[corte:corte*2])
lista_urls.append(df.url[corte*2:corte*3])
lista_urls.append(df.url[corte*3:df.shape[0]])

In [236]:
#lista_urls

In [230]:
#scrapear(lista_urls[0])

In [237]:
%%time
from joblib import Parallel, delayed
results = Parallel(n_jobs=-1)(delayed(scrapear)(urls) for urls in lista_urls)

Wall time: 2min 53s


In [238]:
comentarios = results[0] + results[1] + results[2] + results[3]

In [248]:
import pandas as pd
pd.DataFrame(comentarios).to_csv('./data/comments_infobae_10_notas.csv')

In [60]:
#driver.quit()

In [48]:
#SOBRAS
#creo que hay un tema con el tiempoq ue tarda en renderear la pagina
#element = driver.find_element_by_tag_name('button')
#element.location_once_scrolled_into_view
#driver.find_element_by_xpath("//button").click()
#driver.find_element_by_xpath("//*[contains(text(), 'Cargar')]").click()
#driver.find_element_by_xpath("//button").click()
#button = True
#ver que pasa que no lo hace de corrido?
#todos los comentarios con me gusta, horario y autores,etc
#todo = driver.find_elements_by_class_name('_3-8y')

#solo el texto
#todo = driver.find_elements_by_class_name('_3-8m')
#texto
#todo[2].find_element_by_class_name('_5mdd').text
#autor
#todo[2].find_element_by_class_name('UFICommentActorName').text
#me gusta _2vq9 fsm fwn fcg
#for x in todo[1].find_element_by_class_name('_2vq9').find_elements_by_tag_name('span'):
#    print (x.text)
#fecha
#todo[1].find_element_by_class_name('_2vq9').find_elements_by_tag_name('span')[-1].text
#me gusta (si es . no hay me gusta)
#todo[2].find_element_by_class_name('_2vq9').find_elements_by_tag_name('span')[-3].text